In [1]:
# Prerequisites: python 3.6 or later
import requests
import json
import uuid
import pprint
import datetime
pp = pprint.PrettyPrinter(indent=2)

In [2]:
# This is a convenience method to handle api responses. The main portion of the notebook starts in the the next cell
def handle_api_response(response, print_response=False):
    parsed_response = response.json()

    if print_response:
        pp.pprint({"API Response": parsed_response})
    
    if response.status_code == 200:
        return parsed_response
    elif response.status_code == 400:
        raise Exception("Bad request ^")
    elif response.status_code == 403:
        msg = "Please make sure your request headers include X-Api-Key and that you are using correct url"
        raise Exception(msg)
    else:
        now = datetime.datetime.utcnow().replace(microsecond=0).isoformat()
        msg = f"""\n\n
        ------------------------------------- BEGIN ERROR MESSAGE -----------------------------------------
        It seems our server encountered an error which it doesn't know how to handle yet. 
        This sometimes happens with unexpected input(s). In order to help us diagnose and resolve the issue, 
        could you please fill out the following information and email the entire message between ----- to
        danf@usc.edu:
        1) URL of notebook (of using the one from https://hub.mybinder.org/...): [*****PLEASE INSERT ONE HERE*****]
        2) Snapshot/picture of the cell that resulted in this error: [*****PLEASE INSERT ONE HERE*****]
        
        Thank you and we apologize for any inconvenience. We'll get back to you as soon as possible!
        
        Sincerely, 
        Dan Feldman
        
        Automatically generated summary:
        - Time of occurrence: {now}
        - Request method + url: {response.request.method} - {response.request.url}
        - Request headers: {response.request.headers}
        - Request body: {response.request.body}
        - Response: {parsed_response}

        --------------------------------------- END ERROR MESSAGE ------------------------------------------
        \n\n
        """

        raise Exception(msg)

The Data Catalog API URL

In [22]:
url = 'https://data-catalog.indiana.mint.isi.edu'

In [23]:
resp = requests.get(f"{url}/get_session_token").json()
print(resp)
api_key = resp['X-Api-Key']

request_headers = {
    'Content-Type': "application/json",
    'X-Api-Key': api_key
}

{'X-Api-Key': 'mint-data-catalog:59d40d50-b1d7-4335-809a-fbd4167ee854:3d1bed87-b551-44f0-8537-ed1ac943d9b6'}


When you register datasets or resources, we require you to pass a "provenance_id". This a unique id associated with your account so that we can keep track of who is adding things to the data catalog. For sandboxed interactions with the data catalog api, please use this provenance_id:

In [24]:
provenance_id = "9ef60317-5da5-4050-8bbc-7d6826fee49f"

Step 0: Registering Standard Variables

In order to disambiguate duplicate names, we require that each variable in your dataset to be associated with one or more standard variables. What makes a variable name "standard" is that it is a part of some ontology, so that anyone can examine that ontology and see for themselves semantic meaning of the variable. Most of our current datasets are mapped to standard names defined by the GSN ontology.

In [9]:
standard_variable_defs = {
    "standard_variables": [
        {
            "name": "Application",
            "ontology": "MyOntology",
            "uri": "test1"
        },
        {
            "name": "Class",
            "ontology": "MyOntology",
            "uri": "test2"
        },
        {
            "name": "LoopId",
            "ontology": "MyOntology",
            "uri": "test3"
        },
        {
            "name": "PAPI_L1_TCM",
            "ontology": "MyOntology",
            "uri": "test4"
        },
        {
            "name": "PAPI_L2_TCM",
            "ontology": "MyOntology",
            "uri": "test5"
        },
        {
            "name": "PAPI_L3_TCM",
            "ontology": "MyOntology",
            "uri": "test6"
        },
        {
            "name": "PAPI_TOT_CYC",
            "ontology": "MyOntology",
            "uri": "test7"
        },
        {
            "name": "PAPI_TOT_INS",
            "ontology": "MyOntology",
            "uri": "test8"
        },
        {
            "name": "PAPI_DP_OPS",
            "ontology": "MyOntology",
            "uri": "test9"
        },
        {
            "name": "PAPI_SP_OPS",
            "ontology": "MyOntology",
            "uri": "test10"
        },
        {
            "name": "PAPI_BR_UCN",
            "ontology": "MyOntology",
            "uri": "test11"
        },
        {
            "name": "PAPI_BR_CN",
            "ontology": "MyOntology",
            "uri": "test12"
        },
        {
            "name": "_ITERATIONS",
            "ontology": "MyOntology",
            "uri": "test13"
        },
        {
            "name": "Number Iterations",
            "ontology": "MyOntology",
            "uri": "test14"
        },
        {
            "name": "Total Iterations",
            "ontology": "MyOntology",
            "uri": "test15"
        },
        {
            "name": "Total Array Access",
            "ontology": "MyOntology",
            "uri": "test16"
        },
        {
            "name": "Ratio Reduction Statements",
            "ontology": "MyOntology",
            "uri": "test17"
        },
        {
            "name": "Number Stores",
            "ontology": "MyOntology",
            "uri": "test18"
        },
        {
            "name": "Number Statements",
            "ontology": "MyOntology",
            "uri": "test19"
        },
        {
            "name": "LoopLevel",
            "ontology": "MyOntology",
            "uri": "test20"
        },
        {
            "name": "NumberBitsIteration",
            "ontology": "MyOntology",
            "uri": "test21"
        },
        {
            "name": "Ratio Reduction Statements.1",
            "ontology": "MyOntology",
            "uri": "test22"
        },
        {
            "name": "If Statements",
            "ontology": "MyOntology",
            "uri": "test23"
        },
        {
            "name": "Conftrol Flow Modifiers other than break Statements",
            "ontology": "MyOntology",
            "uri": "test24"
        },
        {
            "name": "While Loop",
            "ontology": "MyOntology",
            "uri": "test25"
        },
        {
            "name": "Number Scalar Dependencies",
            "ontology": "MyOntology",
            "uri": "test26"
        },
        {
            "name": "Number Scalar Dependencies Array",
            "ontology": "MyOntology",
            "uri": "test27"
        },
        {
            "name": "Number IO operations",
            "ontology": "MyOntology",
            "uri": "test28"
        },
        {
            "name": "Number Float Operations",
            "ontology": "MyOntology",
            "uri": "test29"
        },
        {
            "name": "Short Operations",
            "ontology": "MyOntology",
            "uri": "test30"
        },
        {
            "name": "Number Integer Operations",
            "ontology": "MyOntology",
            "uri": "test31"
        },
        {
            "name": "Number of Function Calls Side effect free",
            "ontology": "MyOntology",
            "uri": "test32"
        },
        {
            "name": "Number of Function Calls",
            "ontology": "MyOntology",
            "uri": "test33"
        },
        {
            "name": "DataDependenceFree",
            "ontology": "MyOntology",
            "uri": "test34"
        },
        {
            "name": "AverageSerial",
            "ontology": "MyOntology",
            "uri": "test35"
        },
        {
            "name": "MinimumSerial",
            "ontology": "MyOntology",
            "uri": "test36"
        },
        {
            "name": "MaximumSerial",
            "ontology": "MyOntology",
            "uri": "test37"
        },
        {
            "name": "AverageParallel",
            "ontology": "MyOntology",
            "uri": "test38"
        },
        {
            "name": "MinimumParallel",
            "ontology": "MyOntology",
            "uri": "test39"
        },
        {
            "name": "MaximumParallel",
            "ontology": "MyOntology",
            "uri": "test40"
        }
    ]
}

If request is successful, it will return 'result': 'success' along with a list of registered standard variables and their record_ids. Those record_ids are unique identifiers (UUID) and you will need them down the road to  register variables

In [10]:
resp = requests.post(f"{url}/knowledge_graph/register_standard_variables", 
                    headers=request_headers, 
                    json=standard_variable_defs)

In [13]:
parsed_response = handle_api_response(resp, print_response=True)

{ 'API Response': { 'result': 'success',
                    'standard_variables': [ { 'description': '',
                                              'name': 'Application',
                                              'ontology': 'MyOntology',
                                              'record_id': 'cce3037d-a400-5aae-83f7-2296e0dfc0ed',
                                              'uri': 'test1'},
                                            { 'description': '',
                                              'name': 'Class',
                                              'ontology': 'MyOntology',
                                              'record_id': 'ecfec8c6-07dd-5179-941f-c0c809a06a61',
                                              'uri': 'test2'},
                                            { 'description': '',
                                              'name': 'LoopId',
                                              'ontology': 'MyOntology',
                            

If you need to check if specific standard variables have already been registered in the data catalog, you can search by name and data catalog will return existing records.

In [14]:
nonexistent_name = str(uuid.uuid4())
print(f"This name does not exist: {nonexistent_name}")

search_query = {
    "name__in": ["MaximumParallel", nonexistent_name]
}

resp = requests.post(f"{url}/knowledge_graph/find_standard_variables", 
                                        headers=request_headers,
                                        json=search_query)
parsed_response = handle_api_response(resp, print_response=True)


This name does not exist: 2a173c1f-32cd-4efd-a6fc-7ffca433b8b0
{ 'API Response': { 'result': 'success',
                    'standard_variables': [ { 'description': '',
                                              'id': '71adaa27-f599-5b87-b6c5-3beed65caa51',
                                              'name': 'MaximumParallel',
                                              'ontology': 'MyOntology',
                                              'uri': 'test40'}]}}


Step 1: REGISTER DATASETS

In [15]:
dataset_id = str(uuid.uuid4()) # This is optional; if not given, it will be auto-generated
print(dataset_id)

63783d21-c624-4036-b5dd-be1024fbfcb0


Build datasets definition

In [25]:
dataset_defs = {
    "datasets": [
        {
            "record_id": dataset_id, # Remove this line if you want to create a new dataset
            "provenance_id": provenance_id,
            "metadata": {
                "any_additional_metadata": "this is a test dataset"
            },
            "description": "this is a dummy dataset",
            "name": "DummyData"
        }
    ]
}

In [26]:
resp = requests.post(f"{url}/datasets/register_datasets", 
                                        headers=request_headers,
                                        json=dataset_defs)


parsed_response = handle_api_response(resp, print_response=True)

{ 'API Response': { 'datasets': [ { 'description': 'this is a dummy dataset',
                                    'json_metadata': { 'any_additional_metadata': 'this '
                                                                                  'is '
                                                                                  'a '
                                                                                  'test '
                                                                                  'dataset'},
                                    'name': 'DummyData',
                                    'provenance_id': '9ef60317-5da5-4050-8bbc-7d6826fee49f',
                                    'record_id': '63783d21-c624-4036-b5dd-be1024fbfcb0'}],
                    'result': 'success'}}


Step 2: Register variables

In [41]:
variable_defs = {
    "variables": [
        {
            #"record_id": time_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id,
            "name": "Application",
            "metadata": {
                "units": "name"
                # Can include any other metadata that you want to associate with the variable
            },
            "standard_variable_ids": [
                'cce3037d-a400-5aae-83f7-2296e0dfc0ed'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Class",
            "metadata": {
                "units": "level"
            },
            "standard_variable_ids": [
                'ecfec8c6-07dd-5179-941f-c0c809a06a61'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "LoopId_id",
            "metadata": {
                "units": "name"
            },
            "standard_variable_ids": [
                '1d260717-d9f8-5722-9e41-13bf1179ff4a'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "LoopId",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '1d260717-d9f8-5722-9e41-13bf1179ff4a'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_L1_TCM",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'f430781e-f52a-5625-8e57-f702e2c22e1b'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_L2_TCM",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '1d07822f-6935-5e40-983f-a573300a10e2'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_L3_TCM",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '98756b35-2a97-5680-bbb0-164031839edb'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_TOT_CYC",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '8a46d6f8-6f87-5411-8ee9-01272851ad02'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_TOT_INS",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '6d8af68f-dac2-52d5-909b-02af7189d5c6'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_DP_OPS",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'aafb88a2-72aa-589a-a084-338f1e0d5cc4'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_SP_OPS",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '1595ec14-e55e-5def-b5e2-ea72c7f001bb'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_BR_UCN",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '76d6896d-ddd8-5362-8380-905c808273fa'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "PAPI_BR_CN",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '71192748-e2f7-5608-bc2e-37d152022199'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "_ITERATIONS",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '85de3cec-c34e-585d-a757-f7a06e616da9'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Iterations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'cc6ec01b-2c64-5375-8a42-1d0662dbafa7'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Total Iterations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '4c7123d7-3fb9-59a7-bad2-ae0827e4cc7b'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Total Array Access",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '19654b8a-42d6-552d-9cf6-a443a397b73d'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Ratio Reduction Statements'",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '5a3b0769-544b-5090-af9a-96723c6c0866'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Stores",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '5b71b0f2-33fc-511b-b789-bd80e1be0cc0'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Statements",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'dc8a51bc-f7e1-5409-b61b-26fd20c4e204'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "LoopLevel",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '9a5df59e-45a2-5070-868b-f37b02de897c'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "NumberBitsIteration",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '48e4d361-bcd3-518b-9c1d-64412ad4901f'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Ratio Reduction Statements.1",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '034070ef-0738-5b97-9ec3-141da5b6331a'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "If Statements",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'ab0e5d69-cee0-51de-bab8-87b22c749d26'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Conftrol Flow Modifiers other than break Statements",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'a2250dca-d4cb-5162-ad55-a0466fb7f6b6'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "While Loop",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'cf0f53d4-de30-5c08-bc36-851a50635560'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Scalar Dependencies",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '596aab99-07ef-51eb-8b88-fbb5d8fedf9f'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Scalar Dependencies Array",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'a0d2f0c7-8105-507e-a9e3-55c96a986554'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number IO operations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'a451866c-3650-513c-94e4-499b4edca196'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Float Operations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'a5508091-3822-5e7f-88ab-b528a3617304'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Short Operations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '79fc4e37-be94-5318-beb4-46c41be3828b'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number Integer Operations",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '4b4926ed-a11b-5beb-bce9-3ce29b8e1c4e'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number of Function Calls Side effect free",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '8da7202b-5501-517f-bbff-6296bd3298b7'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "Number of Function Calls",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '90d4832b-6ebb-5405-acdb-43512e3abb29'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "DataDependenceFree",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'f4a3fbc6-7647-5eb1-9b2b-d047d0922c36'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "AverageSerial",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '635a0cc3-7d36-5394-8834-2763bf492e54'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "MinimumSerial",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'ec5b9204-675b-52b4-b93e-06a7fb13cc15'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "MaximumSerial",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '67596db9-22eb-566b-afa6-07fbcb801b32'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "AverageParallel",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                'db931026-baf4-5310-a0ab-dac9cb7c50c7'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "MinimumParallel",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '7d66dfe6-6630-5cc7-9000-e48e06068214'
            ]
        },
        {
            #"record_id": temperature_variable_record_id, # If you remove this line, record_id will be auto-generated
            "dataset_id": dataset_id, # from register_datasets() call
            "name": "MaximumParallel",
            "metadata": {
                "units": "reading"
            },
            "standard_variable_ids": [
                '71adaa27-f599-5b87-b6c5-3beed65caa51'
            ]
        }
    ]
}

In [42]:
resp = requests.post(f"{url}/datasets/register_variables", 
                                        headers=request_headers,
                                        json=variable_defs)

parsed_response = handle_api_response(resp, print_response=True)

{ 'API Response': { 'result': 'success',
                    'variables': [ { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                     'json_metadata': {'units': 'name'},
                                     'name': 'Application',
                                     'record_id': '626d950a-916b-416c-bd8e-f2c0c0d54146'},
                                   { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                     'json_metadata': {'units': 'level'},
                                     'name': 'Class',
                                     'record_id': 'd8da25e7-ffea-4c79-b6c2-cc206b7c12d1'},
                                   { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                     'json_metadata': {'units': 'name'},
                                     'name': 'LoopId_id',
                                     'record_id': '90e37649-5b84-41c5-8664-dd72653611e4'},
                     

Step 3: Register resources

Assume that I host my datasets files on https://github.com/luoyu357/ICICLEModel

In [43]:
data_storage_url = "https://github.com/luoyu357/ICICLEModel/raw/da68051118aae4458a4332bf6d7f2ed8ba14a10f/data"

In [44]:
file_1_name = "DummyData.xlsx"

In [45]:
file_1_data_url = f"{data_storage_url}/{file_1_name}"

In [46]:
file_1_data_url

'https://github.com/luoyu357/ICICLEModel/raw/da68051118aae4458a4332bf6d7f2ed8ba14a10f/data/DummyData.xlsx'

In [47]:
file_1_record_id = str(uuid.uuid4())

In [48]:
file_1_temporal_coverage = {
    "start_time": "2018-01-01T00:00:00",
    "end_time": "2018-01-01T23:59:59"
}

In [49]:
spatial_coverage = {
    "type": "BoundingBox",
    "value": {
        "xmin": 33.9605286,
        "ymin": -118.4253354,
        "xmax": 33.9895077,
        "ymax": -118.4093589
    }
}

In [50]:
resource_defs = {
    "resources": [
        {
            "record_id": file_1_record_id,
            "dataset_id": dataset_id,
            "provenance_id": provenance_id,
            "variable_ids": [
                '626d950a-916b-416c-bd8e-f2c0c0d54146',
                'd8da25e7-ffea-4c79-b6c2-cc206b7c12d1',
                '90e37649-5b84-41c5-8664-dd72653611e4',
                '0da476aa-7299-4b3d-9791-90863c1996fe',
                'ed73dfb7-c2c2-4f22-a285-e59224b19a1b',
                'eba52d34-74e7-4e81-9717-dc45156f4de9',
                '6b6e74b5-b165-48b7-83de-9cb6d3aa0f11',
                '68368b06-8592-47ea-b77e-90ea3dade841',
                '896a0621-4e5e-4d82-9b91-36f85db11ca2',
                'a05b8c99-91cc-47fa-97af-eb111f03cda2',
                '12274273-4c20-4bb2-ad0d-6f636f8ba343',
                'e2f19f59-e004-486a-8ec2-a938ceaeafd1',
                'b2940317-0a3e-4b6b-8dc0-8a59889bba5b',
                '98d1477e-c8b3-4007-80fa-b49b28ae5ba4',
                '9dca713f-de58-482c-808b-7a4e1cd5ddbf',
                '3adba75e-043b-462f-8a66-5ddc04e23de9',
                'afc776b8-26f3-4fab-9b1d-79b60b1add85',
                '6886cdc7-3725-4c84-bf2b-a4127960a15e',
                '6a32d873-1bfb-44ce-9709-f82917dea43c',
                '5cd58cad-fece-4f7e-bc0b-e348dc25e852',
                'c3d02760-a9e2-463b-98c5-478e34149a97',
                'b44af7f1-b88c-42e5-aeca-f3e02c5a9147',
                '18d97988-dee0-4348-a637-34f01935ccdc',
                '153fda0e-f842-4b99-884b-c521c2bbfd69',
                'a7e16ef3-4d9f-4527-ac05-6ebf0b92d320',
                'ac6a0058-bf1c-45f6-a926-e4ddf17fd53f',
                'a5dbefa9-79f9-43ca-b856-ae3db609d5df',
                'e4ac51c6-b6fc-460f-b978-9f564583ba57',
                'fa781976-1c4d-418d-8e9b-05ff22e9b63d',
                '8519a0e0-fd2f-4838-bf1b-b797f0eabf6e',
                '1832cddc-16e8-4c91-8156-b354b0e1409f',
                '5bc91e44-6a0c-4c84-b3d0-ba52e9f725f7',
                '7d06c37a-268d-487f-95d8-e09be87d6efb',
                '37d01285-a3e0-4b4d-a9a9-f631397a3e95',
                'cbcf2e97-2441-4fab-b638-3ee5dfc63805',
                'd66ab3f8-ebef-4a84-a54c-64ea2f91fedf',
                '12e8e880-e6a2-4d16-b6a5-746a49503e99',
                '46b914b0-8574-45e7-880f-798bd63f50e9',
                '5a94dee2-c6b0-462f-92b6-1a0403b2295c',
                '805d8664-235a-4886-ad3d-08d9d40a5ba9',
                '65de4ef7-b852-488b-bc46-788b3f556f32'    
            ],
            "name": file_1_name,
            "resource_type": "csv",
            "data_url": file_1_data_url,
            "metadata": {
                "spatial_coverage": spatial_coverage,
                "temporal_coverage": file_1_temporal_coverage
            },
            "layout": {}
        }
    ]
}

In [51]:
resp = requests.post(f"{url}/datasets/register_resources", 
                                        headers=request_headers,
                                        json=resource_defs)


parsed_response = handle_api_response(resp, print_response=True)

{ 'API Response': { 'resources': [ { 'data_url': 'https://github.com/luoyu357/ICICLEModel/raw/da68051118aae4458a4332bf6d7f2ed8ba14a10f/data/DummyData.xlsx',
                                     'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                     'json_metadata': { 'spatial_coverage': { 'type': 'BoundingBox',
                                                                              'value': { 'xmax': 33.9895077,
                                                                                         'xmin': 33.9605286,
                                                                                         'ymax': -118.4093589,
                                                                                         'ymin': -118.4253354}},
                                                        'temporal_coverage': { 'end_time': '2018-01-01T23:59:59',
                                                                               'start_time': '2

Query Dataset

In [52]:
id = '63783d21-c624-4036-b5dd-be1024fbfcb0'

In [54]:
search_query_id = {
    "dataset_ids__in": [id]
}

resp = requests.post(f"{url}/datasets/find",
                     headers=request_headers,
                     json=search_query_id).json()

if resp['result'] == 'success':
    pp.pprint(resp)

{ 'datasets': [ { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                  'dataset_metadata': { 'any_additional_metadata': 'this is a '
                                                                   'test '
                                                                   'dataset',
                                        'dataset_description': 'this is a '
                                                               'dummy dataset',
                                        'dataset_spatial_coverage': {}},
                  'dataset_name': 'DummyData'}],
  'result': 'success'}


Query Dataset Variable

In [56]:
search_query_variable = {
    "dataset_id": id
}

resp = requests.post(f"{url}/datasets/dataset_variables",
                     headers=request_headers,
                     json=search_query_variable).json()


pp.pprint(resp)

{ 'dataset': { 'variables': [ { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                'standard_variables': [ { 'standard_variable_description': '',
                                                          'standard_variable_id': 'cce3037d-a400-5aae-83f7-2296e0dfc0ed',
                                                          'standard_variable_name': 'Application',
                                                          'standard_variable_ontology': 'MyOntology',
                                                          'standard_variable_uri': 'test1'}],
                                'variable_id': '626d950a-916b-416c-bd8e-f2c0c0d54146',
                                'variable_metadata': {'units': 'name'},
                                'variable_name': 'Application'},
                              { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
                                'standard_variables': [ { 'standard_variable_description': '

Query Dataset Resource

In [55]:
search_query_resource = {
    "dataset_id": id
}

resp = requests.post(f"{url}/datasets/dataset_resources",
                     headers=request_headers,
                     json=search_query_resource).json()


pp.pprint(resp)

{ 'dataset': { 'dataset_id': '63783d21-c624-4036-b5dd-be1024fbfcb0',
               'resources': [ { 'resource_created_at': '2023-08-11 03:32:16',
                                'resource_data_url': 'https://github.com/luoyu357/ICICLEModel/raw/da68051118aae4458a4332bf6d7f2ed8ba14a10f/data/DummyData.xlsx',
                                'resource_id': '55db29fb-5fb5-4de5-9036-b70de41bf046',
                                'resource_metadata': { 'spatial_coverage': { 'type': 'BoundingBox',
                                                                             'value': { 'xmax': 33.9895077,
                                                                                        'xmin': 33.9605286,
                                                                                        'ymax': -118.4093589,
                                                                                        'ymin': -118.4253354}},
                                                       'temporal_co